### 라이브러리 선언

In [19]:
import os
from glob import glob
import shutil
import cv2 as cv
import numpy as np
from tqdm import tqdm

# For parse json|
import json
import pprint

# For drawing
import random
import matplotlib.pyplot as plt
import pylab
%matplotlib inline

#### 최종 데이터셋 구조
- avikus-data  
 ㄴ images  
 ㄴ labels  

- YAML
   - class 0: boat
   - class 1: buoy

기존 데이터셋 문제점은 이름이 중복되는 부분이 있다는 것이다. 따라서 prefix로 추가해야할 것이다

***val은 UK가 포함된 폴더를 모두 넣고, 나머지는 train을 넣도록 한다***

### 입력 이미지 및 출력 관련 폴더 선언

In [20]:
# 세그멘테이션 라벨링 파일이 포함된 폴더
labeling_folder = '../../avikus-data/labeling'
rawimg_folder = '../../avikus-data/rawdata/'
include_classes = ["boat", "buoy"] # 파싱하고자 하는 데이터셋

train_image_folder = '../../avikus-data/images/train'
train_label_folder = '../../avikus-data/labels/train'
val_image_folder = '../../avikus-data/images/val'
val_label_folder = '../../avikus-data/labels/val'

# The length of each row does not have to be equal.
# Each segmentation label must have a minimum of 3 xy points: <class-index> <x1> <y1> <x2> <y2> <x3> <y3>

class2label = { name:i for i, name in enumerate(include_classes) }

In [21]:
pprint.pprint(class2label)

{'boat': 0, 'buoy': 1}


In [22]:
# 랜덤 컬러
def random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

In [23]:
for root, dirs, files in tqdm(os.walk(labeling_folder)):
    if 'json' in dirs:
        fd = os.path.join(root, 'json')
        
        for json_file in glob(os.path.join(fd, "*.json")):
            with open(json_file, "r") as file:
                try:
                    data = json.load(file)
                    
                    # 확인용 디버그 print
                    # print(f"Loaded JSON data from {json_file}:")
                    # pprint.pprint(data)
                    
                    # 사용할 키
                    keys = ["objects", "class_name", "annotation", "x", "y"]
                    
                    json_file_basename_without_extension = os.path.splitext(os.path.basename(json_file))[0]
                    image_file_name = json_file_basename_without_extension + ".jpg"
                    image_subfolder_name = json_file.split(os.path.sep)[-3]
                    
                    # 실제 이미지 파일 경로 파싱
                    image_file_path = os.path.join(rawimg_folder, image_subfolder_name, image_file_name)

                    # 이미지 형식
                    if not os.path.exists(image_file_path):
                        print(f"The file at {image_file_path} does not exist.")
                        
                        image_file_name = json_file_basename_without_extension + ".png"
                        image_file_path = os.path.join(rawimg_folder, image_subfolder_name, image_file_name)
                        
                        if not os.path.exists(image_file_path):
                            print(f"The file at {image_file_path} does not exist.")
                            
                            break
                    
                    image_array = cv.imread(image_file_path)
                    image_array = cv.cvtColor(image_array, cv.COLOR_BGR2RGB)

                    # normalize를 위한 원본 이미지 사이즈 parse
                    image_height, image_width = image_array.shape[:2]
                    
                    draw_img = image_array.copy()
                    
                    # Replace 'labeling' with 'draw_image' in the path
                    save_draw_image_path = os.path.splitext(json_file.replace('labeling', 'draw_image'))[0] + '.jpg'
                    
                    elements = json_file.split(os.path.sep)
                    
                    # UK
                    if 'UK' in json_file:
                        # 이미지에 중복된 이름들이 있어서, prefix를 unique하게 설정
                        # elements[-2]는 'json'이라서 제외
                        save_image_path = os.path.splitext(os.path.join(val_image_folder, '_'.join([elements[-4], elements[-3], elements[-1]])))[0] + '.jpg'
                        # 공백 없애기 
                        save_image_path = save_image_path.replace(" ", "")
                        
                        save_label_path = os.path.splitext(os.path.join(val_label_folder, '_'.join([elements[-4], elements[-3], elements[-1]])))[0] + '.txt'
                        # 공백 없애기 
                        save_label_path = save_label_path.replace(" ", "")
                    else:
                        save_image_path = os.path.splitext(os.path.join(train_image_folder, '_'.join([elements[-4], elements[-3], elements[-1]])))[0] + '.jpg'
                        save_image_path = save_image_path.replace(" ", "")
                        
                        save_label_path = os.path.splitext(os.path.join(train_label_folder, '_'.join([elements[-4], elements[-3], elements[-1]])))[0] + '.txt'
                        save_label_path = save_label_path.replace(" ", "")
                    
                    # 이미지 저장
                    # Create the directory if it doesn't exist
                    if not os.path.exists(os.path.dirname(save_image_path)):
                        os.makedirs(os.path.dirname(save_image_path))
                        print(f"The folder {os.path.dirname(save_image_path)} was created.")
                
                    cv.imwrite(save_image_path, cv.imread(image_file_path))
                    
                    print(f'Done. Image saved to {save_image_path}')
                    
                    # 라벨 normalize 후 저장
                    # Create the directory if it doesn't exist
                    if not os.path.exists(os.path.dirname(save_label_path)):
                        os.makedirs(os.path.dirname(save_label_path))
                        print(f"The folder {os.path.dirname(save_label_path)} was created.")
                        
                    # json 파싱
                    if keys[0] in data:
                        for object in data[keys[0]]:
                            cls_name = object[keys[1]]
                            
                            if cls_name in include_classes:
                                annos = object[keys[2]]
                                color = random_color()
                                
                                category_id = class2label[cls_name]
                                
                                # 객체 단위
                                for obj_annos in annos:    
                                    polygon = []
                                    # 객체 안에 최외곽선을 포함한 Hole이 존재할 수 있음                            
                                    for obj_anno in obj_annos:
                                        for pts in obj_anno:
                                            x, y = pts[keys[3]], pts[keys[4]]
                                            polygon.append([x, y])

                                    normalized_polygon = [[x / image_width, y / image_height] for x, y in polygon]

                                    # Convert category_id to a string and format the normalized_polygon
                                    category_id_str = str(category_id)
                                    normalized_polygon_str = ' '.join([f'{x:.6f} {y:.6f}' for x, y in normalized_polygon])

                                    # Construct the line to be written to the file
                                    line = f'{category_id_str} {normalized_polygon_str}\n'

                                    # Write the line to the file
                                    with open(save_label_path, 'a') as file:
                                        file.write(line)
                    
                except json.JSONDecodeError as e:
                    print(f"Error loading JSON file {json_file}: {e}")
                    

0it [00:00, ?it/s]

The folder ../../avikus-data/images/train was created.
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h46m20s~00h46m35s_230511_ws_left.mp4.mp4_image_00000003.jpg
The folder ../../avikus-data/labels/train was created.
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h46m20s~00h46m35s_230511_ws_left.mp4.mp4_image_00000004.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h46m20s~00h46m35s_230511_ws_left.mp4.mp4_image_00000005.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h46m20s~00h46m35s_230511_ws_left.mp4.mp4_image_00000006.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h46m20s~00h46m35s_230511_ws_left.mp4.mp4_image_00000007.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h46m20s~00h46m35s_230511_ws_left.mp4.mp4_image_00000001.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h46m20s~00h4

3it [00:00,  6.81it/s]

Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h46m20s~00h46m35s_230511_ws_left.mp4.mp4_image_00000008.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_UK_data_record_stbd_berth_3_0m39s_video_right.mp4_image_00000010.jpg
The folder ../../avikus-data/labels/val was created.
Done. Image saved to ../../avikus-data/images/train/batch2_UK_data_record_stbd_berth_3_0m39s_video_right.mp4_image_00000020.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_UK_data_record_stbd_berth_3_0m39s_video_right.mp4_image_00000003.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_UK_data_record_stbd_berth_3_0m39s_video_right.mp4_image_00000014.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_UK_data_record_stbd_berth_3_0m39s_video_right.mp4_image_00000018.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_UK_data_record_stbd_berth_3_0m39s_video_right.mp4_image_00000012.jpg
Done. Image saved to ../../avikus-data/images

8it [00:01,  7.23it/s]

...

338it [01:23,  4.65it/s]

Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h32m30s~00h33m00s_230511_ws_blindright.mp4.mp4_image_00000011.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h01m45s~00h02m45s_230511_ws_left.mp4.mp4_image_00000010.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h01m45s~00h02m45s_230511_ws_left.mp4.mp4_image_00000020.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h01m45s~00h02m45s_230511_ws_left.mp4.mp4_image_00000027.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h01m45s~00h02m45s_230511_ws_left.mp4.mp4_image_00000003.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h01m45s~00h02m45s_230511_ws_left.mp4.mp4_image_00000014.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h01m45s~00h02m45s_230511_ws_left.mp4.mp4_image_00000018.jpg
Done. Image saved to ../../avikus-data/images/train/batc

342it [01:25,  4.02it/s]

Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h01m45s~00h02m45s_230511_ws_left.mp4.mp4_image_00000007.jpg
Done. Image saved to ../../avikus-data/images/train/batch2_wangsan_data_00h01m45s~00h02m45s_230511_ws_left.mp4.mp4_image_00000022.jpg


KeyboardInterrupt: 